# Identify Out of Order Lines

In [ ]:
from glob import glob
from scipy.optimize import least_squares
import pickle
import seaborn as sns

from waveCal import *

## Gather Files

In [ ]:
# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
# ThAr
thar_files = glob('/mnt/home/lzhao/ceph/thar5*/ThAr_*.fits')
thid_files  = glob('/mnt/home/lzhao/ceph/thid5*/ThAr_*.thid')
thar_files, thar_times = sortFiles(thar_files, get_mjd=True)
thar_files = thar_files[1:] # First file is from before LFCs
thar_times = thar_times[1:]
thid_files = sortFiles(thid_files) [1:]
num_thar_files = len(thar_files)

## Identify Lines

In [ ]:
def buildLineList(file_list,file_times,
                  order_range=range(45,76),
                  line_cutoff=0.5, file_cutoff=0.5,verbose=False):
    ### Gather calibration information
    # Find all observed lines in each order and their wavlengths
    if verbose:
        print('Finding all observed modes')
    orders, names, waves = buildLineDB(file_list, order_list=order_range)

    # Find x-values of observed lines
    if verbose:
        print('Finding line center for each mode')
    x_values = getLineMeasures(file_list, orders, names)
    
    
    ### Vetting
    # Find where there is no line information
    x_values[x_values < 1] = np.nan # This will throw a warning
    
    # Get rid of bad lines
    good_lines = np.mean(np.isnan(x_values),axis=0) < line_cutoff
    # Trim everything
    names  = names[good_lines]
    orders = orders[good_lines]
    waves  = waves[good_lines]
    x_values = x_values[:,good_lines]
    if verbose:
        num_good = np.sum(good_lines)
        num_total = good_lines.size
        print('{} of {} lines cut ({:.3}%)'.format(
            (num_total - num_good),num_total,
            (num_total - num_good)/num_total*100))
    
    # Get rid of bad files
    good_files = np.mean(np.isnan(x_values),axis=1) < file_cutoff
    # Trim everything
    x_values = x_values[good_files]
    exp_list = file_list[good_files]
    file_times = file_times[good_files]
    if verbose:
        num_good = np.sum(good_files)
        num_total = good_files.size
        print('{} of {} files cut ({:.3}%)'.format(
            (num_total - num_good),num_total,
            (num_total - num_good)/num_total*100))
        print('Files that were cut:')
        print(file_list[~good_files])
    
    line_dict = {}
    line_dict['names'] = names
    line_dict['orders'] = orders
    line_dict['waves'] = waves
    line_dict['x_values'] = x_values
    line_dict['times'] = file_times
    line_dict['files'] = exp_list
    line_dict['bad_pix'] = np.isnan(x_values)
    return line_dict

In [ ]:
lfc_lines = buildLineList(ckpt_files, lfc_times, verbose=True)

In [ ]:
thar_lines = buildLineList(thid_files, thar_times, verbose=True)

## Find Order of Lines

In [ ]:
lfc_left = np.zeros_like(lfc_lines['x_values'],dtype=bool)
lfc_right = np.zeros_like(lfc_lines['x_values'],dtype=bool)
for m in np.unique(lfc_lines['orders']):
    ord_mask = lfc_lines['orders']==m
    wave_sort = np.argsort(lfc_lines['waves'][ord_mask])
    for i,exp in enumerate(lfc_lines['x_values']):
        exp_sort = exp[ord_mask][wave_sort]
        exp_diff = np.diff(exp_sort)
        lfc_left[i,ord_mask] = np.insert(np.logical_or(exp_diff>0,np.isnan(exp_diff)),0,False)
        lfc_right[i,ord_mask] = np.append(np.logical_or(exp_diff>0,np.isnan(exp_diff)),False)

In [ ]:
thar_left = np.zeros_like(thar_lines['x_values'],dtype=bool)
thar_right = np.zeros_like(thar_lines['x_values'],dtype=bool)
for m in np.unique(thar_lines['orders']):
    ord_mask = thar_lines['orders']==m
    wave_sort = np.argsort(thar_lines['waves'][ord_mask])
    for i,exp in enumerate(thar_lines['x_values']):
        exp_sort = exp[ord_mask][wave_sort]
        exp_diff = np.diff(exp_sort)
        thar_left[i,ord_mask] = np.insert(np.logical_or(exp_diff>0,np.isnan(exp_diff)),0,False)
        thar_right[i,ord_mask] = np.append(np.logical_or(exp_diff>0,np.isnan(exp_diff)),False)

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Good LFC Lines')
plt.xlabel('Line Number')
plt.ylabel('Percentage of Good Lines')

# Draw order boundaries
old = 0
for i,o in enumerate(lfc_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

nan_lines = np.sum(np.isnan(lfc_lines['x_values']),axis=0)

plt.plot((np.sum(np.logical_and(lfc_left,lfc_right),axis=0))/num_lfc_files,
         'ko',label='both')
plt.plot((np.sum(lfc_left,axis=0))/num_lfc_files,'c.',label='left')
plt.plot((np.sum(lfc_right,axis=0))/num_lfc_files,'r.',label='right')
plt.legend(loc=3)
plt.savefig('./Figures/191220_lfcOrder.png')

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Good ThAr Lines')
plt.xlabel('Line Number')
plt.ylabel('Percentage of Good Lines')

# Draw order boundaries
old = 0
for i,o in enumerate(thar_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

plt.plot((np.sum(np.logical_and(thar_left,thar_right),axis=0))/num_thar_files,
         'ko',label='both')
plt.plot((np.sum(thar_left,axis=0))/num_thar_files,'c.',label='left')
plt.plot((np.sum(thar_right,axis=0))/num_thar_files,'r.',label='right')
plt.legend(loc=3)
plt.savefig('./Figures/191220_tharOrder.png')